In [1]:
import numpy as np #importing libraries
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import glob

In [2]:
start = "arochime-invpfbB0329+54_32768chan3ntbin" #setting strings to make calling files easier/shorter, since most of them share the same name
fold = "foldspec_2018-08-16T10:"
icount = "icount_2018-08-16T10:"
end = ".000+30.000000000000004sec"

In [3]:
def half_min_test(num): #function to determine whether time is in half a minute or whole minute
    if num == 30:
        return '30'
    else:
        return '00'

In [4]:
pt = np.loadtxt('pulse_times.txt').T #loading file of times of pulses
pt = pt.astype(int)

In [5]:
def load(filenames,i): #function for loading in files in a for loop
    data_fold = np.load(start+fold+str(filenames[0,i])+":"+half_min_test(filenames[1,i])+end+".npy")    #loading data from folded pulses
    data_count = np.load(start+icount+str(filenames[0,i])+":"+half_min_test(filenames[1,i])+end+".npy") #loading data from icount file
    return data_fold, data_count ## shape = (3,32768,512,4), (3,32768,512)

In [6]:
def normalize(data_fold,data_count):                               #function for normalizing raw data
    norm_data = data_fold/data_count.reshape(data_count.shape+(1,))
    return norm_data                                               #shape = (3,32768,512,4)

In [7]:
def meandiv_2(norm_data,i):                                                  #function for dividing normalized data by mean
                                                                           #of normalized data for a given time along the
                                                                           #frequency axis to 'clean' it up
    mean_div = norm_data[i,:,:]/norm_data[i,:,:].mean(1, keepdims=True)
    return mean_div                                                        #shape = (32768,512)

In [8]:
def remove_baseline(mean_div):          #function for removing baseline from a given file
    f_summed = np.sum(mean_div,axis=0)  #summing cleaned, normalized data along the frequency axis
                                        #shape = (512)
    base = np.mean(f_summed)            #determining baseline as mean of the weights of the phases along the newly\
                                        #created frequency-summed-over array
    sigma = np.std(f_summed)            #determining standard deviation of the weights of the phases along the newly\
                                        #created frequency-summed-over array
    data_new_baseline = f_summed - base #determining data without baseline by subtracting baseline from newly created\
                                        #frequency-summed-over array
    return data_new_baseline, sigma, base

In [9]:
def find_pulse(data_new_baseline,sigma):
    lower = int(len(data_new_baseline))     #initializing lower bound as maximum possible value
    upper = 0                               #initializing upper bound as minimum possible value
    for i in range(len(data_new_baseline)): #looping over all phases in baseline-corrected phase data
        if (data_new_baseline[i] > sigma):  #conditional to check the positions in the array of the phase values which
                                            #have a weight greater than a standard deviation
            if lower>i:                     #determining lowest and higest values of the phase weights higher than one
                                            #standard deviation to determine start and stop of phase
                lower = i
            elif upper<i:
                upper = i
    return lower, upper

In [10]:
n = len(sorted(glob.glob('test_outputs/*.npy')))/2 #determining the number of files in the folder
n = int(n) #converting number of files into an integer so it can be itterated over

bounds = np.zeros((int(n*3),3)) #n number of files * 3 time bins per file length

All_Folded_Data = np.zeros((n,3,32768,512,4)) #defining arrays to store all loaded data
All_Count_Data = np.zeros((n,3,32768,512))

In [11]:
%%time
#master = np.zeros((len(data_fold[0,:,0,0]),n*3))
k = 0
for j in range(0,n,1):
    folded = np.load(start+fold+str(pt[0,j])+":"+half_min_test(pt[1,j])+end+".npy")    #loading data from folded pulses
    count = np.load(start+icount+str(pt[0,j])+":"+half_min_test(pt[1,j])+end+".npy") #loading data from icount file
    All_Folded_Data[j] = folded #loading data into array containing all loaded data such that each file can be called upon later without need to load it into the notebook again
    All_Count_Data[j] = count
    normalized = normalize(folded,count) #normalizing data
    summed = normalized[:,:,:,0]+normalized[:,:,:,3] #adding XX and YY polarization axis
    for i in range(len(summed[:,0,0])): #iterating over time bins
        
        mean_data = meandiv_2(summed,i) #dividing my mean
        no_baseline_data,stdev,base = remove_baseline(mean_data) #removing baseline data and determining baseline and standard deviation
        lb,ub = find_pulse(no_baseline_data,stdev) #finding lower and upper bound for each pulse
        
        bounds[k] = (lb,ub,base) #storing the upper and lower bound values in an array contaning bounds for pulse for each time axis
        k = k+1

CPU times: user 1min 47s, sys: 3min 13s, total: 5min 1s
Wall time: 5min 7s


In [12]:
lower_bound = int(np.floor(np.mean(bounds[:,0]))) #determining average of lower and upper bounds to determine constant bounds
upper_bound = int(np.ceil(np.mean(bounds[:,1])))
print(upper_bound)
print(lower_bound)

458
434


In [13]:
def pulse_profile_2(data): #defining function to calculate weighted average of data clean data
    upper = upper_bound
    lower = lower_bound
    
    phase_only = np.mean(data,axis=0,keepdims=True) #averaging data along time axis, keeping dimensions
    phase_only = np.sum(phase_only[0,:,:],axis=0) #averaging data along the frequency axis, result is a 1D array with weighted phase values
    
    profile = np.zeros_like(phase_only) #creating an array of 0's length equal to the number of phase bins
    profile[lower:upper] = phase_only[lower:upper] #assinging pulse reigon of phase to on pulse profile array
    
    off_gate = np.zeros_like(phase_only) #creating an array of 0's length equal to the number of phase bins
    off_gate[100:200] = 1 #assigning 1 to off pulse profile array over relativelly large phase interval not containing pulse
    
    weighted_avg_on = np.sum(data*profile,axis=2)/np.sum(profile) #calculating weighted average of on pulse data
    
    weighted_avg_off = np.sum(data*off_gate,axis=2)/np.sum(off_gate) #calculating weighted average of off pulse data
    
    cleaned_data = weighted_avg_on/weighted_avg_off - 1 #cleaning up data

    return cleaned_data

In [14]:
def mask_bands_2(data):
    for i in range(len(data[0,:])):
        if (i >= 10500 and i <= 11700) or (i >= 13500 and i <= 14000) or (i >= 15000 and i <= 15500) or (i >= 26500 and i <= 29250): #conditional for noisy frequency bands
            data[:,i] = 0           #weighing down the RFI frequencies
    return data

In [ ]:
%%time
k=0
h=0
for j in range(0,n,1):
    folded_data = All_Folded_Data[j] #getting raw folded data from master array
    count_data = All_Count_Data[j] #getting raw count data from master array
    normalized = normalize(folded_data,count_data) #normalizing raw data
    summed_data = (normalized[:,:,:,0] + normalized[:,:,:,3]) #summing XX and YY polarization axis
    
    clean = pulse_profile_2(summed_data) #calculating weighted average of data and cleaning data

    masked_pulse_data = mask_bands_2(clean) #masking noisy frequency channels

    if (j == 0):
        master_stack = masked_pulse_data #assigning data to master array for concatenation along time axis
    else:
        master_stack = np.concatenate((master_stack,masked_pulse_data)) #concatenating data to master array along time axis
            

In [ ]:
print(master_stack.shape) #result is a 1D array of time,frequency with shape (number of time bins per file * number of data files, number of frequency bins)

In [ ]:
%%time
plt.figure(figsize=(14,14))
plt.imshow(master_stack.T,aspect='auto') #transposing concatenated data to ensure that x and y axis are correctly orientated on the plot
plt.xlabel('Time', size=20)
plt.ylabel('Frequency', size=20)
plt.colorbar()
plt.savefig('../Dynamic/Dynamic_Spectra_test_18.png')